In [81]:
Max_Player = 900
import requests
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [82]:
from bs4 import BeautifulSoup


def extract_salary_table(data):
    soup = BeautifulSoup(data, 'html.parser')
    table = soup.find('table', class_='table-striped table-responsive table-hover career sngl_pl_tble')
    #rows = table.find_all('tr')

    
    players_info = []

    for row in table.find_all('tr'):
        tds = row.find_all('td')
        if len(tds) != 3:
            continue
        
        year_retained_td, team_name_td, salary_td = tds
        
        year_retained_text = year_retained_td.get_text().strip()
        year = year_retained_text.split()[0]
        retained = ('(Retain)' or '(Retained)') in year_retained_text
        
        team_name = team_name_td.get_text().strip()
        
        salary_text = salary_td.get_text().strip()
        currency, salary = salary_text.split()
        
        players_info.append([year, retained, team_name, salary])

    
    '''

    results = []
    for row in table.find_all('tr'):
        cols = row.find_all('td')
        print(cols)
    
        for col in cols:
            
            
            year = col.text.split()[0]
            retained = '(Retained)' in col.text.split()[1]
            
            team = col[1].text.strip()
            salary = col[2].text.strip()
            if year and team and salary:
                results.append([year, retained, team, salary])

    '''
    

    

    return players_info


def extract_information(text):
    start = text.find('Batting ')
    end = text.find('<li><p>Nationality ', start) + len('<li><p>Nationality ')
    if start != -1 and end != -1:
        txt = text[start:end]
# Extract batting type
        batting_type = txt.split('Batting <span>')[1].split('</span>')[0]

# Extract bowling type
        bowling_type = txt.split('Bowling <span>')[1].split('</span>')[0]

        return batting_type, bowling_type
    else:
        return "Information not found"

In [83]:
test_link = 'https://moneyball.insidesport.in/single-player.php?id=25'

# build webscraper to get data from the link using requests
def final_scraper(link):
    r = requests.get(link)

    name = r.text.split('<h4>')[1].split('</h4>')[0]
# remove all white spaces from name and print
    name = name.replace(' ','')
    # also remove all special characters
    name = ''.join(e for e in name if e.isalnum())

    batting_type, bowling_type = extract_information(r.text)

    table = extract_salary_table(r.text)



    return name, batting_type, bowling_type, table




In [84]:
name,batting,bowling,table = final_scraper(test_link)

In [86]:
# the number at the end of temp link should be a variable
# that increments by 1 each time

# create a list of links to scrape
skip_list = [26,236,250,259,294,334,346,353,430,466,497,517,554,556,555,557,553,552,551,560]
player_df = []
for i in range(1,3):

    if i in skip_list:
        continue
    temp_link = 'https://moneyball.insidesport.in/single-player.php?id=' + str(i)
    name, batting_type, bowling_type, salary = final_scraper(temp_link)
    player_df.append([name, batting_type, bowling_type, salary])
    # print every 10 iterations
    if i % 5 == 0:
        print(i)


player_df = pd.DataFrame(player_df, columns=['Name', 'Batting Type', 'Bowling Type', 'Salary'])
# save the player_df to a csv file
player_df.to_csv('player_df.csv', index=False)
    

In [88]:

player_df.Salary[1]

[['2008', False, 'Chennai Super Kings', '14,056,000']]